In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
driver = webdriver.Edge()

In [3]:
results=[]

In [4]:
def url_content(url):
    driver.get(url)
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    return soup

In [5]:
def get_product_url(soup):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    product_link= []
    start_link = "https://www.flipkart.com"
    for item in soup.find_all('a',href=True, attrs={'class':'_1fQZEK'}):
        rest_link = item['href']
        product_link.append(start_link+rest_link)
    return product_link


In [6]:
def get_product_name(soup):
    product_name= []
    try:
        name=soup.find('span',attrs={'class':"B_NuCI"}).text
        product_name.append(name)
    except:
        name = "No product name available"
        product_name.append(name)
    #product_name = "Product Name"
     
    return product_name


In [7]:
def get_product_price(soup):
    product_price = []
    try:
        price = soup.find('div',attrs={'class':"_30jeq3 _16Jk6d"}).text
        product_price.append(price)
    except:
        price = "NO price available"
        product_price.append(price)
    
    return product_price


In [8]:
def get_product_ratings(soup):
    product_ratings = []
    try:
        ratings = soup.find('div', attrs = {'class' : '_3LWZlK'}).text
        product_ratings.append(ratings)
    except:
        ratings = "No rating available"
        product_ratings.append(ratings)
    
    return product_ratings


In [9]:
def get_specifications(soup):
    for item in soup.find_all("div", class_="_2418kt"):
            item_specifications = [spec.text for spec in item.find_all("li", class_="_21Ahn-")]
            specifications.append(item_specifications)
    supported_apps = [spec[0].split(': ')[1] for spec in specifications]
    operating_system = [spec[1] for spec in specifications]
    resolution = [spec[2] for spec in specifications]
    sound_output = [spec[3] for spec in specifications]
    refresh_rate = [spec[4] for spec in specifications]
    return (supported_apps,operating_system,resolution,sound_output,refresh_rate)

In [10]:
url = "https://www.flipkart.com/search?q=smart+tv&sid=ckf%2Cczl&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_5_na_na_ps&as-pos=1&as-type=HISTORY&suggestionId=smart+tv%7CTelevisions&requestId=ac4665a8-c1ef-462b-9fb8-e5913908cc39"
#driver.get(url)

In [12]:
data = pd.DataFrame({"Product_Url" : [], "Product Name" : [], "Price" : [], "Ratings" : []})

In [13]:
url_contents=url_content(url)

In [14]:
product_link = get_product_url(url_contents)

In [15]:
data["Product_Url"] = product_link

In [ ]:
df = pd.DataFrame()
def assign_to_dataframe(product_content):
    product_names = get_product_name(product_content)
    product_prices = get_product_price(product_content)
    product_ratings= get_product_ratings(product_content)
    df =pd.DataFrame({
           "Product Name": product_names,
            "Price": product_prices,
            "Ratings" : product_ratings
        })

    return df
for product in range(len(data)):
    product_url = data["Product_Url"].iloc[product]
    product_content = url_content(product_url)
    df1 = assign_to_dataframe(product_content)
    df = pd.concat([df, df1], axis=0, ignore_index=True, sort=False)
df